# Setup

In [ ]:
from getpass import getpass

API_KEY = getpass("Please input your AlphaVantage API Key: ")

Please input your AlphaVantage API Key: ··········


## Report Generation Function


In [ ]:
import pandas as pd
from pandas import read_csv
import plotly.graph_objects as go
import requests


def generate_stocks_report(symbol="NFLX"):

    base_url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": symbol,
        "apikey": API_KEY,
        "datatype": "csv"
    }

    try:

        response = requests.get(base_url, params=params)
        response.raise_for_status()

        data = pd.read_csv(response.url)


        expected_columns = ['timestamp', 'open', 'high', 'low', 'close', 'adjusted_close', 'volume', 'dividend_amount', 'split_coefficient']
        if not all(column in data.columns for column in expected_columns):
            print("OOPS, couldn't find that stock. Please check your symbol and try again.")
            return

    except requests.exceptions.RequestException as e:
        print("OOPS, could not find that stock. Please check your symbol and try again.")
        return

    #print(data)


    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['timestamp'],
                         y=data['adjusted_close'],
                         mode='lines',
                         name='Closing Price',
                         line=dict(color='blue')))

    data["50_day_avg"] = data["adjusted_close"].rolling(window=50, min_periods=0).mean()
    fig.add_trace(go.Scatter(x=data['timestamp'],
                         y=data['50_day_avg'],
                         mode='lines',
                         name='50-day Moving Avg',
                         line=dict(color='black')))
    fig.update_layout(title=f"Daily Stock Prices ({symbol})",
                  xaxis_title="Date",
                  yaxis_title="Closing Price",
                  yaxis_tickprefix="$",
                  yaxis_tickformat=",",
                  showlegend=False)
    fig.show()



    fig_candlestick = go.Figure(data=[go.Candlestick(x=data['timestamp'],
                                                     open=data['open'],
                                                     high=data['high'],
                                                     low=data['low'],
                                                     close=data['close'])])
    fig_candlestick.update_layout(title=f"Candlestick Chart ({symbol})",
                                  xaxis_title="Date",
                                  yaxis_title="Price",
                                  yaxis_tickprefix="$")
    fig_candlestick.show()


    print("Column Names:", list(data.columns))
    print("Number of Days:", len(data))
    print("Latest Day:", data['timestamp'].iloc[0])
    print("Earliest Day:", data['timestamp'].iloc[-1])
    print("Closing Price on Latest Day: ${:.2f}".format(data['adjusted_close'].iloc[0]))
    print("Closing Price on Earliest Day: ${:.2f}".format(data['adjusted_close'].iloc[-1]))
    print("100-day High Price: ${:.2f}".format(data['high'].max()))
    print("100-day Low Price: ${:.2f}".format(data['low'].min()))

    def to_pct(decimal):
      return "{:.4f}%".format(decimal * 100)


    earliest_close = data['adjusted_close'].iloc[-1]
    latest_close = data['adjusted_close'].iloc[0]
    pct_change = (latest_close - earliest_close) / earliest_close
    print("Percentage Change:", to_pct(pct_change))
    print("50-day Moving Average on Latest Day: ${:.2f}".format(data['50_day_avg'].iloc[0]))


#generate_stocks_report(symbol="NVDA")




## Stocks Data Dashboard

If your function works, we should be able to use it within the context of the dashboard below:

 1. Use the dropdown to select a stock symbol.
 2. Run the cell to generate a report and chart chart of prices over time.

In [ ]:
# @title Stock Selection Form
selected_symbol = "NVDA" # @param ['MSFT', 'GOOGL', 'AAPL', 'NFLX', "SBUX", "TSLA", "DIS", "NVDA"]
generate_stocks_report(selected_symbol)

Column Names: ['timestamp', 'open', 'high', 'low', 'close', 'adjusted_close', 'volume', 'dividend_amount', 'split_coefficient', '50_day_avg']
Number of Days: 100
Latest Day: 2023-10-19
Earliest Day: 2023-05-30
Closing Price on Latest Day: $421.01
Closing Price on Earliest Day: $401.03
100-day High Price: $502.66
100-day Low Price: $373.56
Percentage Change: 4.9814%
50-day Moving Average on Latest Day: $421.01
